In [1]:
# HW9 SQL Alchemy - Surf's Up
import pandas as pd

# Import, inspect and clean the hawaii_measurements file
measures = pd.read_csv("/Users/frimpter/Documents/data_science/ru_datascience_bootcamp/Homework9 SQLAlchemy/hawaii_measurements.csv")

#measures.dtypes
#measures.columns
#measures.shape

measures = measures.dropna(axis=0).reset_index(drop=True)
#measures.shape
measures.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.06,70


In [2]:
# Import, inspect and clean the hawaii_stations file

stations = pd.read_csv("/Users/frimpter/Documents/data_science/ru_datascience_bootcamp/Homework9 SQLAlchemy/hawaii_stations.csv")

#stations.dtypes
#stations.shape
#stations.columns
stations.head(10)


,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [3]:
# Export both cleaned tables to new csv files
measures.to_csv("clean_hawaii_measurements.csv", header=True, index=False)
stations.to_csv("clean_hawaii_stations.csv", header=True)

In [4]:
# Use SQL Alchemy to model table schemas and create a sqlite database for the tables
# Dependencies
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Column, Integer, String, Numeric, Text, Float


In [5]:
# Create an engine to a SQLite database file called hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

# Engine connection
conn = engine.connect()


In [6]:
# Create ORM classes for each table

Base = declarative_base()

class Measurements(Base):
    __tablename__ = "measurements"

    id = Column(Integer, primary_key=True, autoincrement=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

class Stations(Base):
    __tablename__ = "stations"

    station = Column(Text, primary_key=True)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
    
# Create the 
Base.metadata.create_all(engine)

In [7]:
# Read in the new CSV files so we can load them into MySQL

measures_df = pd.read_csv("/Users/frimpter/Documents/data_science/ru_datascience_bootcamp/Homework9 SQLAlchemy/clean_hawaii_measurements.csv")
stations_df = pd.read_csv("/Users/frimpter/Documents/data_science/ru_datascience_bootcamp/Homework9 SQLAlchemy/clean_hawaii_stations.csv")


In [8]:
measures_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.06,70


In [9]:
# Hold the data in objects
measures_data = measures_df.to_dict(orient='records')
stations_data = stations_df.to_dict(orient='records')

#print(stations_data) # Check the smaller file

# Reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

# Make references to the tables as variables
measures_table = sqlalchemy.Table("measurements", metadata, autoload=True)
stations_table = sqlalchemy.Table("stations", metadata, autoload=True)

# Insert the data into the tables
conn.execute(measures_table.insert(), measures_data)
conn.execute(stations_table.insert(), stations_data)


In [20]:
# Test that the insert worked
conn.execute("select * from measurements limit 5").fetchall()
#conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.06, 70)]